### Preprocessing Extreme Weather events

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("../raw_data/Weather_and_Climate_data/extreme_weather_events.csv", engine="python")
print(df.head())         # Show top 5 rows
print(df.info())



FileNotFoundError: [Errno 2] No such file or directory: '../raw_data/Weather_and_Climate_data/extreme_weather_events.csv'

In [46]:
# Clean column names (remove extra spaces, unify case)
df.columns = df.columns.str.strip().str.replace(" ", "_").str.replace("(", "").str.replace(")", "").str.replace("'", "").str.lower()
#remove duplicates and trim strings
df.drop_duplicates(inplace=True)
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


C:\Users\SHYAM PANDIT\AppData\Local\Temp\ipykernel_9236\295919590.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [ ]:
df.replace(['', '..', 'N/A', 'na'], np.nan, inplace=True)
numeric_columns = [
    'magnitude', 'latitude', 'longitude', 'total_deaths', 'no._injured',
    'no._affected', 'no._homeless', 'total_affected',
    'reconstruction_costs_000_us$', 'insured_damage_000_us$',
    'total_damage_000_us$', 'cpi'
]
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Option 1: Fill with median (common for skewed disaster data)
for col in numeric_columns:
    df[col].fillna(df[col].median(), inplace=True)



C:\Users\SHYAM PANDIT\AppData\Local\Temp\ipykernel_9236\3385968364.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
C:\Users\SHYAM PANDIT\AppData\Local\Temp\ipykernel_9236\3385968364.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a co

In [48]:
#Normalize
cost_columns = ['reconstruction_costs_000_us$', 'insured_damage_000_us$', 'total_damage_000_us$']
for col in cost_columns:
    df[col] = df[col] * 1000  # converting to actual USD



In [49]:
# Convert to integers after filling NaNs
df['start_day'] = df['start_day'].fillna(1).astype(int)
df['start_month'] = df['start_month'].fillna(1).astype(int)
df['start_year'] = df['start_year'].fillna(1).astype(int)

df['end_day'] = df['end_day'].fillna(1).astype(int)
df['end_month'] = df['end_month'].fillna(1).astype(int)
df['end_year'] = df['end_year'].fillna(1).astype(int)

# Safely create datetime columns using apply
df['start_date'] = pd.to_datetime(df.apply(lambda row: f"{int(row['start_year'])}-{int(row['start_month'])}-{int(row['start_day'])}", axis=1), errors='coerce')
df['end_date'] = pd.to_datetime(df.apply(lambda row: f"{int(row['end_year'])}-{int(row['end_month'])}-{int(row['end_day'])}", axis=1), errors='coerce')


In [1]:
#Temporal Alignment
df.sort_values(by='start_date', inplace=True)
df.set_index('start_date', inplace=True)
df['start_date'] = pd.to_datetime(df['start_date'])

NameError: name 'df' is not defined

In [51]:
# Georeferencing for Spatial Analysis

import geopandas as gpd
from shapely.geometry import Point

df = df.dropna(subset=['latitude', 'longitude'])  # ensure we have valid coordinates
geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')


In [52]:
# Save cleaned dataset
df.reset_index(inplace=True)
df.to_csv("../processed_data/processed_extreme_weather_events.csv", index=False)


### Preprocessing Glacial Lake data

In [26]:
import geopandas as gpd

gdf = gpd.read_file("../raw_data/Enviromental_data/Glacial_lake_data/nepal_glacial_lakes_2001/data/NepalGlacialLake2001.shp")



In [28]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
import os

# STEP 1: Load Data
gdf_2001 = gpd.read_file("../raw_data/Enviromental_data/Glacial_lake_data/nepal_glacial_lakes_2001/data/NepalGlacialLake2001.shp")
gdf_2011 = gpd.read_file("../raw_data/Enviromental_data/Glacial_lake_data/nepal_glacial_lakes_2011/data/NepalGlacialLake2011.shp")
gdf_2015 = gpd.read_file("../raw_data/Enviromental_data/Glacial_lake_data/data/GL_3basins_2015.shp")

# STEP 2: Clean Column Names
def clean_columns(gdf):
    gdf.columns = gdf.columns.str.strip().str.replace(" ", "_").str.lower()
    return gdf

gdf_2001 = clean_columns(gdf_2001)
gdf_2011 = clean_columns(gdf_2011)
gdf_2015 = clean_columns(gdf_2015)

# STEP 3: Handle Missing Values
def fill_missing(gdf):
    gdf['gl_name'] = gdf['gl_name'].fillna('Unknown')
    gdf['gl_area'] = gdf['gl_area'].fillna(gdf['gl_area'].mean())
    gdf['elevation'] = gdf['elevation'].fillna(gdf['elevation'].mean())
    return gdf

gdf_2001 = fill_missing(gdf_2001)
gdf_2011 = fill_missing(gdf_2011)
gdf_2015['gl_area'] = gdf_2015['area']  # normalize column names
gdf_2015['elevation'] = gdf_2015['elevation']  # already in decimal

# STEP 4: Normalize CRS and Units
for gdf in [gdf_2001, gdf_2011, gdf_2015]:
    gdf.to_crs("EPSG:4326", inplace=True)

# STEP 5: Add Year Column
gdf_2001["year"] = 2001
gdf_2011["year"] = 2011
gdf_2015["year"] = 2015

# STEP 6: Select Common Columns
cols = ['gl_code', 'gl_area', 'elevation', 'year', 'geometry']
gdf_2001 = gdf_2001[cols]
gdf_2011 = gdf_2011[cols]
gdf_2015['gl_code'] = gdf_2015['gl_id']  # rename ID for consistency
gdf_2015 = gdf_2015[cols]

# STEP 7: Combine Datasets
all_years = pd.concat([gdf_2001, gdf_2011, gdf_2015], ignore_index=True)

# STEP 8: Data Lineage
all_years['source'] = all_years['year'].map({
    2001: 'glacial_lakes_2001.shp',
    2011: 'glacial_lakes_2011.shp',
    2015: 'glacial_lakes_2015.shp'
})

# STEP 9: Add Latitude and Longitude from centroid
centroids = all_years.geometry.centroid
all_years['Longitude'] = centroids.x
all_years['Latitude'] = centroids.y

# STEP 10: Validation Checks
invalid_geoms = all_years[~all_years.geometry.is_valid]
print(f"Invalid geometries: {len(invalid_geoms)}")
print(f"Missing Gl_Code: {all_years['gl_code'].isnull().sum()}")
print(f"Negative Areas: {(all_years['gl_area'] < 0).sum()}")

# STEP 11: Export as CSV with WKT Geometry
all_years['geometry'] = all_years['geometry'].apply(lambda geom: geom.wkt)
all_years.to_csv("../processed_data/processed_glacial_lake_data.csv", index=False)


C:\Users\SHYAM PANDIT\AppData\Local\Temp\ipykernel_9556\224108815.py:59: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids = all_years.geometry.centroid


Invalid geometries: 0
Missing Gl_Code: 0
Negative Areas: 0


C:\Users\SHYAM PANDIT\AppData\Local\Temp\ipykernel_9556\224108815.py:70: UserWarning: Geometry column does not contain geometry.
  all_years['geometry'] = all_years['geometry'].apply(lambda geom: geom.wkt)


### Preprocessing Land cover/Land Use data

In [2]:
import rasterio
import pandas as pd
import numpy as np
import os

# Constants
LULC_YEARS = ['2000', '2010', '2022']
INPUT_FOLDER = '../raw_data/Enviromental_data/LULC'
OUTPUT_CSV = '../processed_data/LULC_Combined.csv'
CLASS_MAPPING = {
    1: 'Forest',
    2: 'Shrubland',
    3: 'Grassland',
    4: 'Agriculture',
    5: 'Built-up',
    6: 'Waterbody',
    7: 'Bareland'
}
SAMPLE_RATE = 20  # Sample every 20th pixel to reduce memory usage

def extract_raster_to_csv_sampled(tif_path, year):
    with rasterio.open(tif_path) as src:
        band = src.read(1)
        band = band[::SAMPLE_RATE, ::SAMPLE_RATE]  # Downsample
        transform = src.transform

        rows, cols = np.where((band != src.nodata) & (band != 0))
        xs, ys = rasterio.transform.xy(transform, rows * SAMPLE_RATE, cols * SAMPLE_RATE)
        values = band[rows, cols]

    df = pd.DataFrame({
        "Year": year,
        "Latitude": ys,
        "Longitude": xs,
        "LULC_Code": values,
        "LULC_Label": [CLASS_MAPPING.get(v, 'Unknown') for v in values],
        "Source": "ICIMOD"
    })
    return df

def main():
    all_dfs = []

    for year in LULC_YEARS:
        tif_file = os.path.join(INPUT_FOLDER, f'lulc({year})', 'data', f'lc{year}.tif')
        print(f"Processing {tif_file}...")
        df = extract_raster_to_csv_sampled(tif_file, year)
        all_dfs.append(df)

    combined_df = pd.concat(all_dfs, ignore_index=True)
    combined_df.to_csv(OUTPUT_CSV, index=False)
    print(f"Saved sampled CSV to {OUTPUT_CSV}")

if __name__ == "__main__":
    main()


Processing ../raw_data/Enviromental_data/LULC\lulc(2000)\data\lc2000.tif...
Processing ../raw_data/Enviromental_data/LULC\lulc(2010)\data\lc2010.tif...
Processing ../raw_data/Enviromental_data/LULC\lulc(2022)\data\lc2022.tif...
Saved sampled CSV to ../processed_data/LULC_Combined.csv


In [5]:
import rasterio
import pandas as pd
import numpy as np
import os

# Constants
LULC_YEARS = ['2000', '2010', '2022']
INPUT_FOLDER = '../raw_data/Enviromental_data/LULC'
OUTPUT_CSV = '../processed_data/LULC_Sampled.csv'
CLASS_MAPPING = {
    1: 'Forest',
    2: 'Shrubland',
    3: 'Grassland',
    4: 'Agriculture',
    5: 'Built-up',
    6: 'Waterbody',
    7: 'Bareland'
}
SAMPLE_RATE = 20  # Sample every 20th pixel

def validate_data(df):
    return df[
        (df["Latitude"].between(-90, 90)) &
        (df["Longitude"].between(-180, 180)) &
        (df["LULC_Code"].isin(CLASS_MAPPING.keys()))
    ]

def extract_raster_to_csv_sampled(tif_path, year):
    with rasterio.open(tif_path) as src:
        band = src.read(1)
        band = band[::SAMPLE_RATE, ::SAMPLE_RATE]
        transform = src.transform

        rows, cols = np.where((band != src.nodata) & (band != 0))
        xs, ys = rasterio.transform.xy(transform, rows * SAMPLE_RATE, cols * SAMPLE_RATE)
        values = band[rows, cols]

    df = pd.DataFrame({
        "Year": year,
        "Latitude": ys,
        "Longitude": xs,
        "LULC_Code": values,
        "LULC_Label": [CLASS_MAPPING.get(v, 'Unknown') for v in values],
        "Source": "ICIMOD"
    })

    df = validate_data(df)
    return df

def main():
    all_dfs = []

    for year in LULC_YEARS:
        tif_file = os.path.join(INPUT_FOLDER, f'lulc({year})', 'data', f'lc{year}.tif')
        print(f"Processing {tif_file}...")
        df = extract_raster_to_csv_sampled(tif_file, year)
        all_dfs.append(df)

    combined_df = pd.concat(all_dfs, ignore_index=True)
    combined_df.to_csv(OUTPUT_CSV, index=False)
    print(f"Saved sampled CSV to {OUTPUT_CSV}")

if __name__ == "__main__":
    main()



Processing ../raw_data/Enviromental_data/LULC\lulc(2000)\data\lc2000.tif...
Processing ../raw_data/Enviromental_data/LULC\lulc(2010)\data\lc2010.tif...
Processing ../raw_data/Enviromental_data/LULC\lulc(2022)\data\lc2022.tif...
Saved sampled CSV to ../processed_data/LULC_Sampled.csv


### Preprocessing River Discharge data

In [4]:
import os
import pandas as pd
import re
from glob import glob


In [7]:
def parse_grdc_file(filepath):
    try:
        with open(filepath, 'r', encoding='latin1') as file:
            lines = file.readlines()
    except UnicodeDecodeError:
        with open(filepath, 'r', encoding='ISO-8859-1') as file:
            lines = file.readlines()

    metadata = {}
    data_start = 0

    # Extract metadata
    for idx, line in enumerate(lines):
        if line.startswith("# River:"):
            metadata['River'] = line.split(':', 1)[1].strip()
        elif line.startswith("# Station:"):
            metadata['Station'] = line.split(':', 1)[1].strip()
        elif line.startswith("# Latitude"):
            metadata['Latitude'] = float(line.split(':', 1)[1].strip())
        elif line.startswith("# Longitude"):
            metadata['Longitude'] = float(line.split(':', 1)[1].strip())
        elif line.startswith("# Next downstream station:"):
            metadata['NextStation'] = line.split(':', 1)[1].strip()
        elif line.strip() == "# DATA":
            data_start = idx + 2
            break

    # Read tabular data
    data = pd.read_csv(
        filepath,
        sep=';',
        skiprows=data_start,
        names=['Date', 'Time', 'Original', 'Calculated', 'Flag'],
        na_values=['-999.000'],
        encoding='latin1'
    )

    # Use original value only
    data['Discharge'] = data['Original']
    data['Year'] = pd.to_datetime(data['Date'], errors='coerce').dt.year
    data = data.dropna(subset=['Year'])

    for key, val in metadata.items():
        data[key] = val

    return data[['River', 'Station', 'Latitude', 'Longitude', 'NextStation', 'Year', 'Discharge']]


In [9]:
folder_path = '../raw_data/Enviromental_data/river discharge data'  # Replace with your actual path
all_files = glob(os.path.join(folder_path, '*.txt'))

combined_df = pd.DataFrame()

for file in all_files:
    try:
        river_data = parse_grdc_file(file)
        if not river_data.empty:
            combined_df = pd.concat([combined_df, river_data], ignore_index=True)
        else:
            print(f"Skipped empty data in file: {file}")
    except Exception as e:
        print(f"Failed to parse {file}: {e}")



Skipped empty data in file: ../raw_data/Enviromental_data/river discharge data\2548350_Q_Day.Cmd.txt
Skipped empty data in file: ../raw_data/Enviromental_data/river discharge data\2548450_Q_Day.Cmd.txt
Skipped empty data in file: ../raw_data/Enviromental_data/river discharge data\2548460_Q_Day.Cmd.txt
Skipped empty data in file: ../raw_data/Enviromental_data/river discharge data\2548500_Q_Day.Cmd.txt
Skipped empty data in file: ../raw_data/Enviromental_data/river discharge data\2548550_Q_Day.Cmd.txt
Skipped empty data in file: ../raw_data/Enviromental_data/river discharge data\2548610_Q_Day.Cmd.txt
Skipped empty data in file: ../raw_data/Enviromental_data/river discharge data\2548620_Q_Day.Cmd.txt
Skipped empty data in file: ../raw_data/Enviromental_data/river discharge data\2549235_Q_Day.Cmd.txt
Skipped empty data in file: ../raw_data/Enviromental_data/river discharge data\2549300_Q_Day.Cmd.txt
Skipped empty data in file: ../raw_data/Enviromental_data/river discharge data\2549350_Q_Da

In [ ]:
# Handle missing discharge using group mean (or interpolate)
combined_df['Discharge'] = combined_df.groupby(['River', 'Station'])['Discharge'].transform(
    lambda x: x.fillna(x.mean())
)



In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
combined_df['Discharge_Normalized'] = scaler.fit_transform(combined_df[['Discharge']])


In [12]:
annual_df = combined_df.groupby(['River', 'Station', 'Latitude', 'Longitude', 'NextStation', 'Year']) \
                       .agg({'Discharge': 'mean', 'Discharge_Normalized': 'mean'}) \
                       .reset_index()


In [26]:
schema = {
    "Title" : "Annual Mean Discharge Data",
    "River": "Name of the river",
    "Station": "Measurement station name",
    "Latitude": "Station latitude in decimal degrees",
    "Longitude": "Station longitude in decimal degrees",
    "NextStation": "Next downstream station ID",
    "Year": "Year of observation",
    "Discharge": "Annual mean discharge in m³/s",
    "Discharge_Normalized": "Z-score normalized discharge"
}

# Save schema documentation
with open("processed_data_schema.txt", "w", encoding="utf-8") as f:
    for col, desc in schema.items():
        f.write(f"{col}: {desc}\n")



In [15]:
assert annual_df.isnull().sum().sum() == 0, "Missing values found!"
assert (annual_df['Year'] >= 1900).all(), "Invalid year values"
assert annual_df['Discharge'].min() >= 0, "Discharge cannot be negative"


In [25]:
# Pivot raw and normalized separately
pivot_raw = combined_df.pivot_table(
    index=['River', 'Station', 'Latitude', 'Longitude', 'NextStation'],
    columns='Year',
    values='Discharge'
)

pivot_norm = combined_df.pivot_table(
    index=['River', 'Station', 'Latitude', 'Longitude', 'NextStation'],
    columns='Year',
    values='Discharge_Normalized'
)

# Rename columns for clarity
pivot_raw.columns = [f"Raw_{col}(m³/s)" for col in pivot_raw.columns]
pivot_norm.columns = [f"Norm_{col}(m³/s)" for col in pivot_norm.columns]

# Combine
pivot_both = pd.concat([pivot_raw, pivot_norm], axis=1).reset_index()


In [ ]:
# Sort columns: group raw first, then normalized
raw_cols = sorted([col for col in pivot_both.columns if col.startswith("Raw_")])
norm_cols = sorted([col for col in pivot_both.columns if col.startswith("Norm_")])
main_cols = ['River', 'Station', 'Latitude', 'Longitude', 'NextStation']

# Reorder the DataFrame
final_df = pivot_both[main_cols + raw_cols + norm_cols]

# Export with UTF-8 encoding to handle special characters like ³
final_df.to_csv("../raw_data/Enviromental_data/river discharge data/final_river_discharge.csv", index=False, encoding='utf-8')
print(" Final discharge data saved as 'final_river_discharge.csv'")


✅ Final discharge data saved as 'final_river_discharge.csv'


In [ ]:
import pandas as pd

# Assuming your DataFrame is named 'final_df'
final_df = pd.read_csv("../raw_data/Enviromental_data/river discharge data/final_river_discharge.csv")

# Interpolate missing values in 'Raw' and 'Norm' columns
# This will fill NaN values using the linear interpolation method
final_df[raw_cols] = final_df[raw_cols].interpolate(axis=1, method='linear', limit_direction='both')
final_df[norm_cols] = final_df[norm_cols].interpolate(axis=1, method='linear', limit_direction='both')

# Save the cleaned data back to CSV with interpolation
final_df.to_csv("river_discharge_data_cleaned.csv", index=False, encoding='utf-8')
print(" Interpolated data saved as 'river_discharge_data_cleaned.csv'")


✅ Interpolated data saved as 'final_river_discharge_interpolated.csv'


### Preprocessing  Population data

In [3]:
import pandas as pd

# Load the datasets
precipitation_data = pd.read_csv("../raw_data/Weather_and_CLimate_data/district-precipitation-trends-with-corresponding-significance-levels (1).csv")
weather_events_data = pd.read_csv("../raw_data/Weather_and_CLimate_data/extreme_weather_events.csv")

# Extract relevant columns
precipitation_data = precipitation_data[['Districts', 'Annual_Trend']]
weather_events_data = weather_events_data[['Location', 'Disaster Type']]

# Clean and process the data
# Group weather events by district and count occurrences
weather_events_data['Location'] = weather_events_data['Location'].str.strip()
weather_events_count = weather_events_data.groupby('Location').size().reset_index(name='Extreme_Weather_Events_Count')

# Merge the datasets on district/location
merged_data = pd.merge(precipitation_data, weather_events_count, left_on='Districts', right_on='Location', how='inner')

# Add a placeholder for population distribution (can be replaced with actual data)
merged_data['Population'] = 'Unknown'  # Replace with actual population data if available

# Save the combined data to a new CSV file
output_file = "../raw_data/Socioeconomic_data/population_distribution_data_in_climate_vulnerable_areas.csv"
merged_data.to_csv(output_file, index=False)

print(f"File '{output_file}' has been created successfully.")


File '../raw_data/Socioeconomic_data/population_distribution_data_in_climate_vulnerable_areas.csv' has been created successfully.


### Preprocessing Climate reanalysis data 

In [16]:
import xarray as xr
import numpy as np
# Load the NetCDF file
ds = xr.open_dataset("../raw_data/Weather_and_Climate_data/era5_nepal_temp.nc")
# Select relevant variables
vars_of_interest = ['t2m', 'tp', 'smlt', 'lblt', 'src']
data = ds[vars_of_interest]

# Convert to long-format DataFrame
df = data.to_dataframe().reset_index()

# Drop rows with any NaN values (or use interpolation if time series)
df = df.dropna()

# Optional: check for remaining NaNs
assert not df.isnull().values.any(), "Still has missing values!"

# Convert Kelvin to Celsius
df['t2m'] = df['t2m'] - 273.15

# Normalize each variable (optional - for ML preprocessing)
for var in ['t2m', 'tp', 'smlt', 'lblt', 'src']:
    df[f'{var}_norm'] = (df[var] - df[var].mean()) / df[var].std()

# Rename columns for clarity
df = df.rename(columns={
    'latitude': 'lat',
    'longitude': 'lon',
    'valid_time': 'date',
    't2m': 'temperature_celsius',
    'tp': 'total_precip_m',
    'smlt': 'snow_melt_m',
    'lblt': 'large_scale_snow_melt_m',
    'src': 'snow_runoff_m'
})

# Value range assertions
assert df['temperature_celsius'].between(-80, 60).all(), "Temperature out of range!"
assert df['total_precip_m'].between(0, 1).all(), "Precipitation seems too high!"

# Duplicate checks
assert df.duplicated(subset=['lat', 'lon', 'date']).sum() == 0, "Duplicate spatial entries!"

df.to_csv("../processed_data/processed_era5.csv", index=False)


### Preprocessing Historical temperature and precipatation 

In [19]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# load the CSV
df = pd.read_csv('../raw_data/Weather_and_Climate_data//historic_temp_precipitatio.csv')

# column Names and Strip Whitespace
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

#  Missing Values (Interpolate, then Fill)
df = df.interpolate(method='linear').fillna(method='ffill').fillna(method='bfill')

#  Normalize Numeric Data (excluding geolocation or ID columns if present)
exclude_columns = ['station', 'district', 'date', 'latitude', 'longitude']
numeric_cols = df.select_dtypes(include='number').columns.difference(exclude_columns)
scaler = MinMaxScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

#  Temporal Alignment
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y', errors='coerce')
    df = df.sort_values(by='date')

#  Check for Georeferenced Structure
#  Add or merge latitude/longitude columns if missing
# sumes they are present; otherwise you'd merge from a station metadata file

#  Convert to Long Format (Optional - for easier analysis)
if all(col in df.columns for col in ['date', 'station', 'district']):
    df_long = df.melt(id_vars=['date', 'station', 'district'], var_name='parameter', value_name='value')
    df_long.to_csv('processed_historic_temp_precipitatio.csv', index=False)

#  Save Processed Data
df.to_csv('../processed_data/cleaned_historic_temp_precipitation.csv', index=False)

#  Validation Checks
def validate_dataset(df):
    print("\n--- Data Validation ---")
    print("Null values per column:\n", df.isnull().sum())
    if 'date' in df.columns:
        print("Date Range:", df['date'].min(), "to", df['date'].max())
    invalid = df.select_dtypes(include='number') < 0
    if invalid.any().any():
        print("Warning: Negative values detected in numeric columns.")
    else:
        print("No negative values found. Validation passed.")

validate_dataset(df)


C:\Users\SHYAM PANDIT\AppData\Local\Temp\ipykernel_14972\1121377299.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
C:\Users\SHYAM PANDIT\AppData\Local\Temp\ipykernel_14972\1121377299.py:12: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df = df.interpolate(method='linear').fillna(method='ffill').fillna(method='bfill')
C:\Users\SHYAM PANDIT\AppData\Local\Temp\ipykernel_14972\1121377299.py:12: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.interpolate(method='linear').fillna(method='ffill').fillna(method='bfill')



--- Data Validation ---
Null values per column:
 date           28548
year               0
month              0
district           0
lat                0
lon                0
prectot            0
ps                 0
qv2m               0
rh2m               0
t2m                0
t2mwet             0
t2m_max            0
t2m_min            0
t2m_range          0
ts                 0
ws10m              0
ws10m_max          0
ws10m_min          0
ws10m_range        0
ws50m              0
ws50m_max          0
ws50m_min          0
ws50m_range        0
dtype: int64
Date Range: 1981-01-31 00:00:00 to 2019-12-31 00:00:00
No negative values found. Validation passed.


### Preprocessing Infrastructure mapping in flood-prone regions

In [15]:
import geopandas as gpd

# Load the shapefiles
gdf = gpd.read_file("../raw_data/socioeconomic_data/flood prone/data/nepalvdcindicators.shp")
districts = gpd.read_file("../raw_data/socioeconomic_data/flood prone/data/district.shp")

# Standardize column names (optional)
gdf.columns = gdf.columns.str.strip().str.upper()

# Drop irrelevant or duplicate columns (if any)
gdf = gdf.loc[:, ~gdf.columns.duplicated()]

In [12]:
import pandas as pd
from sklearn.impute import SimpleImputer

# Drop geometry for ML processing
df = gdf.drop(columns='GEOMETRY')

# Identify numeric columns
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

# Simple mean imputation (can customize per column)
imputer = SimpleImputer(strategy='mean')
df[numeric_cols] = imputer.fit_transform(df[numeric_cols])


In [16]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

gdf.to_file("../processed_data/flood_vulnerability.geojson", driver="GeoJSON")
df.to_csv("../processed_data/flood_vulnerability.csv", index=False)



In [37]:
print("Min:", gdf["FLOOD_FREQ"].min())
print("Max:", gdf["FLOOD_FREQ"].max())
print("Unique values:", gdf["FLOOD_FREQ"].unique())


Min: 0.0
Max: 23.0
Unique values: [ 0.  1.  3.  5.  2.  7.  6.  4. 13.  8.  9. 17. 14. 12. 23. 10.]


In [ ]:
# Check for nulls
assert gdf.isnull().sum().sum() == 0, "Missing values found"

# Check for duplicates
assert gdf.duplicated().sum() == 0, "Duplicates found"

# Range checks (example)
assert df['FLOOD_FREQ'].between(0, 10).all(), "FLOOD_FREQ out of expected range"


AssertionError: FLOOD_FREQ out of expected range

### Preprocessing Agriculture yield statistics

In [22]:
import pandas as pd

df = pd.read_csv("../raw_data/Socioeconomic_data/Agriculture Statistics/data/NepalAgriStats_Vegetable.csv")
df.columns = [col.strip() for col in df.columns]  # Clean column names
df['DISTRICT_NAME'] = df['DISTRICT_NAME'].str.title()  # Standardize names


In [23]:
# Check for missing values
print(df.isnull().sum())

# Fill missing values using forward-fill (good for time-series data)
df.fillna(method='ffill', axis=1)


DISTRICT_CODE    0
DISTRICT_NAME    0
VG_P_200304      0
VG_P_200405      0
VG_P_200506      0
VG_P_200607      0
VG_P_200708      0
VG_P_200809      0
VG_P_200910      0
VG_P_201011      0
VG_P_201112      0
VG_P_201213      0
VG_P_201314      0
VG_A_200304      0
VG_A_200405      0
VG_A_200506      0
VG_A_200607      0
VG_A_200708      0
VG_A_200809      0
VG_A_200910      0
VG_A_201011      0
VG_A_201112      0
VG_A_201213      0
VG_A_201314      0
VG_Y_200304      0
VG_Y_200405      0
VG_Y_200506      0
VG_Y_200607      0
VG_Y_200708      0
VG_Y_200809      0
VG_Y_200910      0
VG_Y_201011      0
VG_Y_201112      0
VG_Y_201213      0
VG_Y_201314      0
dtype: int64


C:\Users\SHYAM PANDIT\AppData\Local\Temp\ipykernel_12492\546564371.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', axis=1)


,DISTRICT_CODE,DISTRICT_NAME,VG_P_200304,VG_P_200405,VG_P_200506,VG_P_200607,VG_P_200708,VG_P_200809,VG_P_200910,VG_P_201011,...,VG_Y_200405,VG_Y_200506,VG_Y_200607,VG_Y_200708,VG_Y_200809,VG_Y_200910,VG_Y_201011,VG_Y_201112,VG_Y_201213,VG_Y_201314
0,1,Taplejung,6160,6280.0,6659,5920,6063,6565.0,6080.0,17261.0,...,8698.06,8784.96,8314.61,8362.76,8660.95,7937.34,11928.82,11201.3,10789.88,10789.88
1,2,Panchthar,5850,6943.0,7363,7672,7994,11523.77,11409.65,46577.0,...,8912.71,9012.24,8983.61,8951.85,10779.95,10280.82,13978.69,9891.7,10734.62,10754.18
2,3,Ilam,30000,24916.0,26424,27534,28691,43113.0,44820.0,23694.0,...,9805.59,9904.05,9875.9,9849.3,13318.81,13997.5,15385.71,14135.49,14065.29,14423.76
3,4,Jhapa,50400,52399.0,55567,57901,60330,63312.0,64233.0,101897.0,...,12271.43,12395.05,12358.8,12322.3,12392.25,12352.5,12403.77,16609.67,17073.76,17374.62
4,5,Sankhuwasabha,11647,12082.0,12812,14770,10756,14502.0,16855.0,7203.8,...,10344.18,10450.24,10075.03,9787.08,10924.29,12044.45,14770.97,9438.45,9270.13,10218.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,71,Kailali,21900,28278.0,29989,34852,40500,123000.0,124620.0,45896.0,...,13338.68,13472.15,13561.09,13650.15,13666.67,13846.67,12154.66,15061.63,12112.67,15353.66
71,72,Darchula,2590,2151.0,2281,2377,11319,10894.1,12587.5,22225.61,...,6938.71,7018.46,6991.18,12618.73,12284.73,12621.58,9033.7,12002.73,11039.71,11112.05
72,73,Baitadi,6725,5354.0,5677,8745,8850,10383.75,11459.55,8206.95,...,10237.09,10340.62,10473.05,10172.41,8747.89,9058.93,7233.98,9309.88,12613.72,12228.79
73,74,Dadeldhura,6720,5131.0,5441,7839,7900,6664.0,7184.95,10466.975,...,8249.2,8332.31,13000.0,11987.86,6670.67,6637.37,10546.11,12194.24,12357.58,12288.62


In [24]:
for year in ['200304', '200405', '200506', '200607', '200708', '200809',
             '200910', '201011', '201112', '201213', '201314']:
    df[f'VG_Y_{year}'] = df[f'VG_P_{year}'] / df[f'VG_A_{year}']


In [25]:
years = ['200304','200405','200506','200607','200708','200809',
         '200910','201011','201112','201213','201314']
available_years = sorted(set(col.split('_')[-1] for col in df.columns if col.startswith('VG_P')))
print("Aligned:", available_years == years)


Aligned: True


In [29]:
import geopandas as gpd

# Load geo data (GeoJSON or shapefile)
geo_df = gpd.read_file("../raw_data/Socioeconomic_data/flood prone/data/district.shp")  # or .shp

# Standardize column names
geo_df['DNM'] = geo_df['DNM'].str.title().str.strip()
df['DISTRICT_NAME'] = df['DISTRICT_NAME'].str.title().str.strip()

# Merge using correct district name columns
merged_gdf = geo_df.merge(df, left_on='DNM', right_on='DISTRICT_NAME')


In [31]:
df_without_geometry = merged_gdf.drop(columns=['geometry'])
df_without_geometry.to_csv("../processed_data/processed_agriculture_data.csv", index=False)
merged_gdf.to_file("../processed_data/processed_agriculture_data.geojson", driver="GeoJSON")

### Preprocessing Weather Station Data

In [15]:
import pandas as pd

# Load the raw CSV file
df = pd.read_csv("../raw_data/Weather_and_Climate_data/meteorological-station-of-nepal.csv")

# Clean column names
df.columns = df.columns.str.strip().str.replace(" ", "_").str.replace(".", "", regex=False)

# Remove entirely empty rows or with very few data
df = df.dropna(thresh=3)

# Strip strings and correct known typos
df['District'] = df['District'].str.strip().replace({'Baltadi': 'Baitadi'})


In [16]:
# Impute missing Index No. with -1 (or a placeholder)
df['Index_No'] = df['Index_No'].fillna(-1)

# Impute elevation with median of known elevations
df['Elemeter'] = df['Elemeter'].fillna(df['Elemeter'].median())

# Drop rows where essential data (e.g., Lat/Long) is missing
df = df.dropna(subset=['LatLogdegmin'])


In [17]:
def dms_to_dd(dms_str):
    """Convert deg.min format (e.g., 2941) to decimal degrees"""
    try:
        dms_str = str(dms_str).strip()
        degrees = int(dms_str[:-2])
        minutes = int(dms_str[-2:])
        return round(degrees + minutes / 60, 6)
    except:
        return None

df['Latitude'] = df['LatLogdegmin'].apply(lambda x: dms_to_dd(x) if pd.notnull(x) else None)
df['Longitude'] = df.iloc[:, 7].apply(lambda x: dms_to_dd(x) if pd.notnull(x) else None)


In [18]:
print(df[['AgEstdDate', 'NMS_EstdDate']].head(10))


  AgEstdDate NMS_EstdDate
0        NaN          NaN
1   I MAY 56        26665
2    I MAY73        26696
3   I MAY 56        26665
4   I MAY 56        25720
5        NaN        25965
6        NaN        25965
7        NaN        27061
8        NaN        27912
9   I JUN 56        26665


In [19]:
import re
from datetime import datetime

# Custom parser for 'I MAY 56' → '1956-05-01'
def parse_ag_date(val):
    if pd.isnull(val):
        return pd.NaT
    match = re.search(r'([A-Z]+)\s*(\d{2})', val)
    if match:
        month_str, year_suffix = match.groups()
        try:
            return datetime.strptime(f"{month_str} 19{year_suffix}", "%b %Y")
        except:
            return pd.NaT
    return pd.NaT

# Custom parser for '26665' → assume it's a Julian date (example logic)
def parse_nms_date(val):
    try:
        # Assuming it's a Julian day from 1900-01-01
        base = datetime(1900, 1, 1)
        return base + pd.to_timedelta(int(val), unit="D")
    except:
        return pd.NaT

# Apply custom parsing
df['AgEstdDate'] = df['AgEstdDate'].apply(parse_ag_date)
df['NMS_EstdDate'] = df['NMS_EstdDate'].apply(parse_nms_date)


In [20]:
import geopandas as gpd
from shapely.geometry import Point

# Create geometry points
geometry = [Point(xy) for xy in zip(df['Longitude'], df['Latitude'])]

# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")  # WGS84


In [21]:
# Select and reorder fields
schema_columns = [
    'Station_Name', 'Index_No', 'Basin_Office', 'Types_of_Station',
    'District', 'Latitude', 'Longitude', 'Elemeter',
    'AgEstdDate', 'NMS_EstdDate', 'geometry'
]

# Save to CSV
gdf[schema_columns].to_csv("../processed_data/cleaned_weather_stations.csv", index=False)


In [22]:
# Check coordinate bounds (Nepal approx lat 26°–31°, long 80°–89°)
invalid_coords = df[(df['Latitude'] < 26) | (df['Latitude'] > 31) |
                    (df['Longitude'] < 80) | (df['Longitude'] > 89)]

# Check for duplicates
duplicates = df[df.duplicated(['Station_Name', 'Latitude', 'Longitude'], keep=False)]

print(f"Invalid coordinates found: {len(invalid_coords)}")
print(f"Potential duplicate stations: {len(duplicates)}")


Invalid coordinates found: 1
Potential duplicate stations: 0
